In [213]:
import re
import csv
import unicodedata

file_name="./csv/0_500_union_salida_clasificada.csv"
stop_file="custom_stopwords.txt"    # Nombre del archivo de stopwords.

# Regex para emoticones en texto.
emoticons_str = r"""
(?:
    [:=;]               # Ojos
    [oO\-]?             # Nariz (optional)
    [D\)\]\(\]/\\OpP]   # Bocas
)"""

# Regex para tokenizar correctamente.
regex_str = [
    emoticons_str,
    r'(?:[\w_]+)',                                        # Otras palabras
    r'(?:\S)'                                             # Cualquier otra cosa
]

# Se arman objetos para regular expresions.
tokens_re   = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

# Se carga archivos de STOPWORDS
with open(stop_file, newline='') as file:
    stopwords = file.read().splitlines()

    
# Se abre archivo con tweets y se lo recorre    
with open(file_name, newline='') as csvfile:

    reader = csv.reader(csvfile, delimiter=',', quotechar='\"')
    header = next(reader)
    
    for row in reader:

        tweet = row[1].lower()                    # Se normaliza texto, todo a minusculas.
        tweet = re.sub(r'@[a-z0-9_]+', '', tweet) # Se quitan menciones. @xxxxxxxx
        tweet = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '', tweet)
        #tweet = re.sub(r'\#+[\w_]+[\w\'_\-]*[\w_]+', '', tweet)
        #tweet = re.sub(r'(#)', '', tweet)
        tweet = tweet.translate(str.maketrans('','', '.~¡!-_—#$%¿?:+-/°);(/",*“”‘’'))
        
        # Manejo de emoticones.
        emoticones = [
                      ['😆',' risa'],['😂','risa'],['😱',' asombro'],['🥳',' felicidad'],['💙',' amor'],['😍',' amor'],
                      ['😀',' sonreir'],['👍',' ok'],['🤔',' ok'],['🎊',' piñata'],['🙏',' ojala'],['💪🏻',' fuerza'],
                      ['😡',' enojo'],['😛',' broma'],['😮',' asombro'],['🤮',' desagradable'],['👏🏻',' aplauso'],
                      ['😏',' canchero'],['😩',' decepcion'],['😳',' verguenza'],['😊',' contento'],['😥',' triste'],
                      ['😤',' furioso'],['🖕',' enojo'],['👏',' aplauso'],['💪',' fuerza']
                     ]
        for emoji in emoticones:
            tweet = tweet.replace(emoji[0], emoji[1])

        # Manejo de acentos.
        dict_acentos = [['á','a'],['é','e'],['í','i'],['ó','o'],['ú','u']]
        for acento in dict_acentos:
            tweet = tweet.replace(acento[0], acento[1])
            
        # Remueve letras repetidas y deja una sola.
        for letra in ['a','e','i','o','u','s','c']:
            pattern = letra + '{2,}'
            tweet = re.sub(pattern, letra, tweet)
        
        tweet = tweet.translate(str.maketrans('','', '🥝🐧🐳🖐🛫😑✈🇦🇷🇵🇾👇🙃▶💻►→⬇️😒🙄🔫🔝🔥💀🚫😞🤦‍♂❤❤❤😎👊🤞🏻'))
        tweet = re.sub(r'\d+', '', tweet)         # Se quitan numeros.
        tweet = tweet.strip()
    
        # Tokenizado
        tokens = tokens_re.findall(tweet)

        # Remocion de stopwords
        tokens = [token for token in tokens if token not in stopwords]

        print(tokens)
        